<a href="https://colab.research.google.com/github/nirajlondhe8/aiml/blob/main/exercise/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Step 1: Generate Sample Data for Network Functions
network_functions = ['gNB', 'AUSF', 'UDM', 'UDR', 'HSS', 'SMSC', 'PGW', 'SGW', 'eNB']
n_samples = 1000  # Number of data points per network function

# Initialize an empty dataframe
data = pd.DataFrame()

# Generate sample data for each network function
for nf in network_functions:
    data[f'{nf}_cpu_usage'] = np.random.normal(loc=50, scale=10, size=n_samples)  # Mean 50%, std dev 10%
    data[f'{nf}_memory_usage'] = np.random.normal(loc=30, scale=5, size=n_samples)  # Mean 30%, std dev 5%
    data[f'{nf}_packet_loss'] = np.random.normal(loc=0.5, scale=0.1, size=n_samples)  # Mean 0.5%, std dev 0.1%
    data[f'{nf}_latency'] = np.random.normal(loc=10, scale=2, size=n_samples)  # Mean 10ms, std dev 2ms

# Prepare the dataset (flatten the columns)
X_train = data.values
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)  # Normalize the data
input_dim = X_train.shape[1]

# Step 2: Build the GAN

# Generator
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=100))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(input_dim, activation='tanh'))  # Output layer matches data dimensionality
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=input_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Loss and Optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Initialize Generator and Discriminator
generator = build_generator()
discriminator = build_discriminator()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Step 3: Train the GAN

EPOCHS = 5000
BATCH_SIZE = 128
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

# Define training steps
@tf.function
def train_step(real_data):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_data = generator(noise, training=True)

        real_output = discriminator(real_data, training=True)
        fake_output = discriminator(generated_data, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Training Loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for data_batch in dataset:
            train_step(data_batch)

        # Generate synthetic data after each epoch for visualization
        if (epoch + 1) % 500 == 0:
            print(f'Epoch {epoch+1} completed')
            generate_and_save_data(generator, epoch + 1, seed)

# Function to generate and save synthetic data
def generate_and_save_data(model, epoch, test_input):
    predictions = model(test_input, training=False)

    # Show a few generated synthetic data points (first 5 rows)
    print(f"Synthetic data at epoch {epoch}:")
    print(predictions.numpy()[:5])  # Display first 5 synthetic samples

# Prepare data for training (as TensorFlow Dataset)
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(10000).batch(BATCH_SIZE)

# Train the GAN
train(train_dataset, EPOCHS)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 500 completed
Synthetic data at epoch 500:
[[-0.9999982   0.99959975  0.9999958   0.09571046 -0.8212809   0.99827224
   0.36365035 -0.99999905 -0.999624   -0.9999877  -0.9744219  -0.999073
   0.9997352  -0.9960045  -0.99999857 -0.9999985  -0.12200284 -0.9992536
   0.81963646 -0.9998412  -0.9985361   1.          0.99627    -0.99997354
  -0.16603185  0.5604886  -0.9999579   0.9711556  -0.8013773  -0.9923729
  -0.99964154  0.35662308 -0.99999374 -0.99543303 -1.          0.9859635 ]
 [ 0.7731209  -0.45191684  0.998036    0.9783854  -0.17855535  0.97345996
   0.00179919  0.7948637  -0.96982956 -0.9612942  -0.73685247 -0.948113
   0.9901937   0.99459237 -0.98920685 -0.43077275  0.6319911  -0.90899956
   0.63764966 -0.9943583   0.85877347  0.99766594 -0.6133064  -0.6199436
  -0.9500699   0.9585462  -0.9389383  -0.05017869  0.82420015 -0.97865677
  -0.00542869  0.84137917 -0.8899688  -0.95044696 -0.9631155  -0.11406729]
 [ 0.7648797   0.4423324  -0.91263354 -0.7784782   0.97187024 -0.625